In [1]:
from pprint import pprint

from pvsamlab.system import System 

In [2]:
for modules_per_string in range(25, 31):
    print(f"Running simulation for modules_per_string={modules_per_string}")
    plant = System(modules_per_string=modules_per_string)
    pprint(plant.model_results)
    print("-" * 50)

Running simulation for modules_per_string=25
{'ac_annual': 303004440.6349879, 'voc_max': 1286.0765075683591}
--------------------------------------------------
Running simulation for modules_per_string=26
{'ac_annual': 303004665.958397, 'voc_max': 1337.5195678710936}
--------------------------------------------------
Running simulation for modules_per_string=27
{'ac_annual': 303000036.16055524, 'voc_max': 1388.9626281738279}
--------------------------------------------------
Running simulation for modules_per_string=28
{'ac_annual': 302984319.60066444, 'voc_max': 1440.3848388671872}
--------------------------------------------------
Running simulation for modules_per_string=29
{'ac_annual': 302990759.1831903, 'voc_max': 1491.8271545410153}
--------------------------------------------------
Running simulation for modules_per_string=30
{'ac_annual': 302894452.50494397, 'voc_max': 1543.291809082031}
--------------------------------------------------
